In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_df=pd.read_csv('../input/big-mart-sales-prediction/Train.csv')
test_df=pd.read_csv('../input/big-mart-sales-prediction/Test.csv')

In [ ]:
test1_df=test_df.copy()

In [ ]:
test1_df

In [ ]:
train_df.head()

In [ ]:
train_df.shape,test_df.shape

In [ ]:
train_df.describe()

In [ ]:
train_df.info()

In [ ]:
cat_cols=train_df.select_dtypes([object])
num_cols=train_df.select_dtypes(exclude=[object])

In [ ]:
cat_cols.drop(['Outlet_Identifier','Item_Identifier'],axis=1,inplace=True)

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

In [ ]:
train_df['Item_Weight'].fillna(train_df['Item_Weight'].mean(),inplace=True)
train_df['Outlet_Size'].fillna(train_df['Outlet_Size'].mode()[0],inplace=True)

In [ ]:
test_df['Item_Weight'].fillna(test_df['Item_Weight'].mean(),inplace=True)
test_df['Outlet_Size'].fillna(test_df['Outlet_Size'].mode()[0],inplace=True)

In [ ]:
train_df.drop(['Item_Identifier','Outlet_Identifier'],axis=1,inplace=True)

In [ ]:
test_df.drop(['Item_Identifier','Outlet_Identifier'],axis=1,inplace=True)

In [ ]:
for col in cat_cols.columns:
    print(col)
    print(train_df[col].value_counts())
    print('--------------------------------------------------------------------------')

In [ ]:
train_df[['Item_Fat_Content']].value_counts()

In [ ]:
# Combine fat content
train_df['Item_Fat_Content']=train_df['Item_Fat_Content'].replace({'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'})

In [ ]:
# Reducing values for outlet year
train_df['Outlet Years']=2013-train_df['Outlet_Establishment_Year']

test_df['Outlet Years']=2013-test_df['Outlet_Establishment_Year']



In [ ]:
test1_df['Outlet Years']=2013-test1_df['Outlet_Establishment_Year']

In [ ]:
train_df

In [ ]:
test_df

### Data Visualization

In [ ]:
sns.displot(train_df['Item_Weight'])

In [ ]:
sns.countplot(train_df['Item_Fat_Content'])

In [ ]:
sns.displot(train_df['Item_Visibility'],kind='kde',color='blue')

In [ ]:
train_df['Item_Type'].value_counts()

In [ ]:
sns.countplot(y=train_df['Item_Type'])

In [ ]:
sns.displot(train_df['Item_MRP'],kind='hist')

In [ ]:
sns.countplot(train_df['Outlet_Size'])

In [ ]:
sns.countplot(train_df['Outlet_Size'],hue=train_df['Outlet_Location_Type'])

In [ ]:
fig, ax = plt. subplots(figsize = (9, 7))
sns.countplot(train_df['Outlet_Size'],hue=train_df['Outlet_Type'])

In [ ]:
sns.countplot(train_df['Outlet_Location_Type'])

In [ ]:
fig, ax = plt. subplots(figsize = (9, 7))
sns.countplot(train_df['Outlet_Location_Type'],hue=train_df['Outlet_Type'])

In [ ]:
fig, ax = plt. subplots(figsize = (9, 7))
sns.countplot(train_df['Outlet_Type'])

In [ ]:
fig, ax = plt. subplots(figsize = (9, 7))
sns.barplot(train_df['Outlet_Type'],train_df['Outlet Years'])

In [ ]:
fig, ax = plt. subplots(figsize = (10, 4))
sns.barplot(train_df['Outlet_Type'],train_df['Outlet Years'],hue=train_df['Outlet_Type'])

In [ ]:
sns.displot(train_df['Item_Outlet_Sales'],kind='kde')

In [ ]:
sns.countplot(train_df['Outlet_Establishment_Year'])

In [ ]:
fig, ax = plt. subplots(figsize = (13, 7))
sns.countplot(train_df['Outlet_Establishment_Year'],hue=train_df['Outlet_Location_Type'])

In [ ]:
fig, ax = plt. subplots(figsize = (13, 7))
sns.countplot(train_df['Outlet_Establishment_Year'],hue=train_df['Outlet_Type'])

In [ ]:
fig, ax = plt. subplots(figsize = (13, 7))
sns.countplot(train_df['Outlet_Establishment_Year'],hue=train_df['Outlet_Size'])

In [ ]:
#Checking for correlation
corr=train_df.corr()
corr

In [ ]:
sns.heatmap(corr,annot=True,cmap='tab20b_r')

In [ ]:
train_df.drop('Outlet_Establishment_Year',axis=1,inplace=True)

In [ ]:
test_df.drop('Outlet_Establishment_Year',axis=1,inplace=True)

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
train_df

### Encoding the data

In [ ]:
#Encoding the data
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
list_cat=list(cat_cols.columns)
train_df[list_cat] =train_df[list_cat].apply(le.fit_transform)


In [ ]:
test_df[list_cat] =test_df[list_cat].apply(le.fit_transform)

In [ ]:
train_df

In [ ]:
test_df

### Model Building

In [ ]:
x=train_df.drop('Item_Outlet_Sales',axis=1)
y=train_df['Item_Outlet_Sales']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.model_selection import KFold,cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

### Decision Tree Regressor

In [ ]:
dt_model=DecisionTreeRegressor()  # We normalize our data for better results
dt_model.fit(x_train,y_train)

y4_pred=dt_model.predict(x_test)


result4=cross_val_score(dt_model,x_train,y_train,cv=KFold(10))


print(result4)
print('---------------------------')
print('CV :',np.mean(result4))
print('---------------------------')
print('MSE:',mean_squared_error(y4_pred,y_test))
print('---------------------------')
print('MAE :',mean_absolute_error(y4_pred,y_test))
print('---------------------------')
print('RMSE:',np.sqrt(mean_squared_error(y4_pred,y_test)))
print('---------------------------')
print('R2:',r2_score(y4_pred,y_test))

### Random forest Regressor

In [ ]:
rf_model=RandomForestRegressor()  # We normalize our data for better results
rf_model.fit(x_train,y_train)

y5_pred=rf_model.predict(x_test)


result5=cross_val_score(rf_model,x_train,y_train,cv=KFold(10))


print(result5)
print('---------------------------')
print('CV :',np.mean(result5))
print('---------------------------')
print('MSE:',mean_squared_error(y5_pred,y_test))
print('---------------------------')
print('MAE :',mean_absolute_error(y5_pred,y_test))
print('---------------------------')
print('RMSE:',np.sqrt(mean_squared_error(y5_pred,y_test)))
print('---------------------------')
print('R2:',r2_score(y5_pred,y_test))

### Hyperparameter Tuning

### Randomized CV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators=[200,400,600,800,1000,1200]

max_features=['auto', 'sqrt','log2']

max_depth = [int(x) for x in np.linspace(10, 1000,10)]

min_samples_split = [2, 5, 10,14]

min_samples_leaf = [1, 2, 4,6,8]

random_grid={'n_estimators':n_estimators,
            'max_features':max_features,
            'max_depth':max_depth,
            'min_samples_split':min_samples_split,
            'min_samples_leaf':min_samples_leaf}
            #'criterion':['mse','mae']}

In [ ]:
print(random_grid)

In [ ]:
rf1=RandomForestRegressor()
rf_randomized=RandomizedSearchCV(estimator=rf1,param_distributions=random_grid,n_iter=100,cv=3,verbose=2,
                               random_state=100,n_jobs=-1)

In [ ]:
rf_randomized.fit(x_train,y_train)

In [ ]:
rf_randomized.best_params_

In [ ]:
rf_final=rf_randomized.best_estimator_

In [ ]:
y_pred_random=rf_final.predict(x_test)

In [ ]:
print('---------------------------')
print('MSE:',mean_squared_error(y_test,y_pred_random))
print('---------------------------')
print('MAE :',mean_absolute_error(y_test,y_pred_random))
print('---------------------------')
print('RMSE:',np.sqrt(mean_squared_error(y_test,y_pred_random)))
print('---------------------------')
print('R2:',r2_score(y_test,y_pred_random))

In [ ]:
predictions = rf_final.predict(test_df)

In [ ]:
output = pd.DataFrame({'Item_Identifier': test1_df.Item_Identifier, 'Outlet_Identifier':test1_df.Outlet_Identifier,'Item_Outlet_Sales':predictions})
output.to_csv('final_submission.csv', index=False)
print("Your submission was successfully saved!")